# A Web Service of Time Series of Earth Observation Data

## Context: The e-sensing project

The e-sensing project is about developing new ways to extract information on land use and land cover change from big Earth Observation data sets. We address the following scientific question: 

***How can we use e-science methods and techniques to improve the extraction of land use and land cover change information from big Earth Observation data sets in an open and reproducible way?*** 

To answer this, our project is building a new generation of knowledge platform for handling big geospatial data. We're conceiving, building, and deploying a new type of knowledge platform for accessing, processing and analyzing big Earth Observation data.

We assume that decades of satellite images can be effectively organized into a data structure which can be efficiently queried and processed using the array data model. 

<img src="img/datacube.png" alt="A data cube of satellite images is as an array" title="data cube" height="300" width="400" />

Our approach is to put data and analysis together to help scientists to do larger, longer, and faster research on land use and land cover change.

<img src="img/architecture.png" alt="e-sensing project's architecture " title="e-sensing architecture" height="300" width="400" />

The "e-sensing" project is supported by __[FAPESP](http://bv.fapesp.br/pt/auxilios/89598/e-sensing-analise-de-grandes-volumes-de-dados-de-observacao-da-terra-para-informacao-de-mudancas-de/)__, under the  __[e-science program](http://www.fapesp.br/8436)__ .

For additional information, please visits us at:

- our  __[official page](http://esensing.org/)__ 
- our code repository at __[github](https://github.com/e-sensing/)__
- our project at __[research gate](https://www.researchgate.net/project/e-sensing-big-earth-observation-data-analytics-for-land-use-and-land-cover-change-information-wwwesensingorg)__


## Web Time Series Service

The Web Time Series Service (WTSS) is a lightweight web service for handling time series data from remote sensing imagery. It exposes three operations:
- ```list_coverages```: get the list of available coverages.
- ```describe_coverage```: get metadata about a specific coverage.
- ```time_series```: get a time series for a given location and time interval.

WTSS is developed and maintained by the National Institute for Space Research of Brazil (__[INPE](http://www.inpe.br/)__), where we have a WTSS server instance running. It can be accessed in the address www.dpi.inpe.br/tws .

For additional details such as the source code and the WTSS clients for Python, R, Javascript and C++ go to the __[WTSS repository](https://github.com/e-sensing/tws/tree/master/src/tws/wtss)__. 

If you are interested on setting your own WTSS server, you can contact us through the __[e-sensing](http://esensing.org/)__ links provided above.

## WTSS for Python

The __[python client](https://github.com/e-sensing/wtss.py)__ for WTSS enables users to retrieve time series of Earth observation data for specific locations, using a few lines of code.
 WTSS easily integrates with Python analysis libraries such as __[numpy](http://www.numpy.org)__, __[scipy](https://www.scipy.org)__, __[pandas](http://pandas.pydata.org)__ and __[matplotlib](https://matplotlib.org)__.

### WTSS: List coverages

This operation gets a list of the data sets hosted in a WTSS server. In the example below, we import the WTSS module and then create a WTSS object to query and print the list of available data sets in the server.

In [ ]:
# WTSS python client: Access to data & metadata
from wtss import wtss

# connect to e-Sensing server
w = wtss("http://www.dpi.inpe.br/tws")

# print the available data sets
cv_list = w.list_coverages()
for cv_name in cv_list["coverages"]:
    print(cv_name)

### WTSS: Describe coverage

This operations enables users to explore the details of a data set in the WTSS server. Below, we ask WTSS for the details of a coverage. Then we format and print the WTSS's reponse.

In [ ]:
# explore a WTSS data set
cv_scheme = w.describe_coverage("mod13q1_512")

# format response
print("ARRAY: {}".format(cv_scheme["name"]))
print("\nDESCRIPTION:\n" + str(cv_scheme['description']) + " - " + str(cv_scheme['detail']))
print("\nDIMENSIONS:{}\n".format(cv_scheme['dimensions']))
print("\nSPATIAL EXTENT:{}\n".format(cv_scheme['spatial_extent']))
print("\nCRS:{}\n".format(cv_scheme['crs']['wkt']))
print("\nTIMELINE:\nFirst five: {}...\nLast  five: ...{}".format(cv_scheme['timeline'][0:5], cv_scheme['timeline'][-5:]))

print("\nATTRIBUTES:")

for att in cv_scheme['attributes']:
    print("{}: {}. Type: {}".format(att['name'], att['description'], att['datatype']))

### WTSS: Time series

This operation retrieves a time series of the provided point. Here, we ask WTSS for some vegetation indexes, then we create *pandas* series out of them, and finally we put the series together into a *pandas* data frame.

In [ ]:
import pandas as pd

# a location that we are interested in observing land dynamics
latitude = -14.919100049
longitude = -59.11781088

# get time series of a point
ts = w.time_series("mod13q1_512", ("ndvi", "evi"), latitude, longitude)

# build a data frame made of vegetation indexes (considering the scale factor)
ndvi = pd.Series(ts["ndvi"], index = ts.timeline)/10000
evi  = pd.Series(ts["evi"],  index = ts.timeline)/10000

vidf = pd.DataFrame({'ndvi': ndvi, 'evi': evi})
vidf[0:5]

Let's show the choosen location on a map.

In [ ]:
from tsmap import *

location = {'lon': longitude, 'lat': latitude}
map = createTSMap(location, vidf, 4)
map

## WTSS and Python Data Analysis Library

### Data Visualization

Python provides tools for scientific data visualization. Below, we take advantage of the integration between *pandas* and *matplotlib* in order to plot our vegetation indexes.

In [ ]:
%matplotlib inline

import matplotlib
from cycler import cycler

matplotlib.style.use('ggplot')

# Updating default matplotlib colors
colors = cycler(u'color', [u'#74c476',u'#6baed6',u'#d62728', u'#ff7f0e', u'#756bb1'])
matplotlib.rcParams['axes.prop_cycle'] = colors

# Time series visualization
fig, ax = matplotlib.pyplot.subplots(figsize = (15, 5))
ax.plot()
vidf['ndvi'].plot()
vidf['evi'].plot()
ax.legend()
fig.autofmt_xdate()

### Filter

#### Line fitting

A simple way to reveal coarse trends in time series is to adjust a straight line through the data. In the code below, we have a function to fit lines which we use in our time series. Then we plot the vegetation indexes along the adjusted lines.

In [ ]:
from linearmodel import *

# fit a line to the vegetation indexes
vidf['ndvi_lm'] = fitline(vidf['ndvi'])
vidf['evi_lm'] = fitline(vidf['evi'])

# plot
fig, ax = matplotlib.pyplot.subplots(figsize = (15, 5))
ax.plot()
vidf['ndvi'].plot()
vidf['evi'].plot()
vidf['ndvi_lm'].plot()
vidf['evi_lm'].plot()
ax.legend()
fig.autofmt_xdate()
vidf[0:5]

#### Fourier decomposition

Fourier series analysis allow us to decompose time series into a sum of waves represented by periodic functions. These functions have properties such as amplitude, wavelength, and frequency. In time series analysis, it is accepted that high frequencies are associated with noise. Therefore, in order to diminish noise we need to remove high frequencies from our time series. The use of Fourier series to estimate vegetation phenology was addresed by Atkinson [[Atkinson2012]](#references).

In the example below, we use our implementation of the Fourier filter function which takes as parameter a time series and the number of low frequencies to keep.

In [ ]:
from fourier import *

# filter the vi
vidf['ndvi_ff'] = fourierfilter(vidf['ndvi'], 25)
vidf['evi_ff'] = fourierfilter(vidf['evi'], 45)

# plot
fig, ax = matplotlib.pyplot.subplots(figsize = (15, 5))
ax.plot()
vidf['ndvi'].plot()
vidf['evi'].plot()
vidf['ndvi_ff'].plot()
vidf['evi_ff'].plot()
ax.legend()
fig.autofmt_xdate()
vidf[0:5]

#### Whittaker smoothing filter

Whitakker filter is obtained by a linear combination of time series nearest neighbors points measures [[Eilers2003]](#references). This filter produces good results for smoothing time-series of satellite data in order to estimate vegetation phenology as reported in  [[Atkinson2012]](#references).

In [ ]:
from whittaker import *

# filter the vi
vidf['ndvi_wf'] = pd.Series(whittaker_filter(ndvi), index = ts.timeline)
vidf['evi_wf']  = pd.Series(whittaker_filter(evi), index = ts.timeline)

# plot
fig, ax = matplotlib.pyplot.subplots(figsize = (15, 5))
ax.plot()
vidf['ndvi'].plot()
vidf['evi'].plot()
vidf['ndvi_wf'].plot()
vidf['evi_wf'].plot()
ax.legend()
fig.autofmt_xdate()
vidf[0:5]

#### Kalman filter

The Kalman filter aims to separate time series from noise. It is an iterative algorithm on which the outputs of one iteration are the inputs for the next one. In this way, the filter successively improves its estimations of the true value of a time series. 

In the example below, we estimate the initial parameters for the filter from the time series itself. Then we compute the Kalman filter and plot the smoothed vegetation indexes.

In [ ]:
from kalman import *

# filter the vi
vidf['ndvi_kf'] = pd.Series(kalmanfilter(ndvi), index = ts.timeline)
vidf['evi_kf']  = pd.Series(kalmanfilter(evi), index = ts.timeline)

# plot
fig, ax = matplotlib.pyplot.subplots(figsize = (15, 5))
ax.plot()
vidf['ndvi'].plot()
vidf['evi'].plot()
vidf['ndvi_kf'].plot()
vidf['evi_kf'].plot()
ax.legend()
fig.autofmt_xdate()
vidf[0:5]

### Classification

#### Dynamic Time Warping

Dynamic Time Warping (DTW) is pattern matching algorithm. It relies on a shape-based distance function that sequentially warps the time dimension in order to find the best match --- the minimum the distance --- between two time series: The pattern and the sample series. Below we show how to classify time series using DTW.

##### Patterns

Our patterns are idealized one-year time-series of the [Forest](Forest) and the [Cerrado](https://en.wikipedia.org/wiki/Cerrado) regions in Brazil.  These patterns were obtained using a [Generalized Additive Model](https://en.wikipedia.org/wiki/Generalized_additive_model) over a large amount of selected time series.

Below we show how to read and plot the aforementioned patterns.

In [ ]:
from dtw import *
from tools import *

# open the pattern file
patterns_ts = pd.read_json("examples/patterns.json", orient='records')
# update timeline type from str to datetime
patterns_ts["timeline"] = pd.to_datetime(patterns_ts["timeline"])
plot_time_series(patterns_ts)

##### Samples

We have a file of sample locations which we would like to classify using the patterns listed above. For the sake of this example, we already know that these locations belong to either the Forest or Cerrado region. These ten locations are in the Brazilian state of Mato Grosso and they were verified on the field.

Below we read the file with the sample locations.

In [ ]:
# read sample file
samples = pd.read_csv("examples/samples.csv")
samples

Now, we get the time series of MODIS data of these locations. We do this in the background using WTSS python client.

In [ ]:
# wtss_get_time_series is implemented in 'tools.py'
samples_ts = wtss_get_time_series(samples)

# rescale vegetation index to -1.0~1.0 range
samples_ts["ndvi"] *= 0.0001
samples_ts["evi"] *= 0.0001

samples_ts[0:5]
plot_time_series(samples_ts)

##### Classify

It is time to classify the sample locations using our patterns. We do this by computing the DTW distance from each pattern to each sample. Then we assign to each sample the name of the pattern with the minimum DTW distance.

To achieve this, we use the code below.

In [ ]:
# classify using DTW
classification = classifier_1nn(patterns_ts, samples_ts)

# print the classification rersults
classification

##### Classification results

The results above prove that DTW and our patterns do a good job. We managed to correctly classify 9 out of ten time series. However, the sample location number 3 is incorrectly classified as it is Cerrado but it was assigned to the Forest label.

To find what happened, we plot the Forest and Cerrado patterns along the time series of the sample location number three. We can see there how this sample doesn't fit very well to either of the two patterns.

In [ ]:
# let see what happened with sample #3
plot_time_series(pd.concat([samples_ts[samples_ts["id"].isin([3])], patterns_ts]))

## Final remarks

We introduced the Web Time Series Service (WTSS), a light weight Web Service of time series of Earth observation data. Through examples and code, we show how the WTSS is used and integrated to Python's scientific libraries such as NumPy, SciPy and Pandas. Therefore, we demonstrated how WTSS fits into the analytic work flow of Earth Observation Scientists.

What are the constraints of WTSS? What is missing?
- So far, our WTSS implementation only provides data from MODIS __[MOD13Q1](https://lpdaac.usgs.gov/dataset_discovery/modis/modis_products_table/mod13q1)__
- The WTSS interface is still simple, we need to extend the service in order to serve complex applications
- Regarding Map Algebra, WTSS enables local operations. We need to extend it to focal, zonal and global operations.
- WTSS is about time series analysis. But we still need to see cubes of satellite data as images. In other words, we need the equivalent of Web Mapping Server, Web Feature Service, and Web Coverage Server for big Earth observation data.
- Our Array database cluster (SciDB) is slow for retrieving images and it does not have native support for the spatio-temporal metadata associated with Earth observation imagery

Finally, we would like to invite you to explore some results of the **e-sensing** project by clicking __[here](http://terrabrasilis.info/composer/E-SENSING)__. We are working on new features of WTSS and also a new Web Service for processing large amounts of Earth observation data.

## Source code

The examples introduced here rely on open source python modules. Our code is available here:
- Map
  - [tsmap.py](./tsmap.py) Display maps in Jupyter notebooks.
- Filter
  - [linearmodel.py](./linearmodel.py) Fit linear models to time series.
  - [fourier.py](./fourier.py) Filter time series using the Fast Fourier Transformation.
  - [kalman.py](./kalman.py) Filter time series using the Kalman filter.
  - [whittaker.py](./whittaker.py) Filter time series using the whittaker smoother.
- Classification. DTW relies in the following modules: 
  - [dtw.py](./dtw.py) Implementation of Dynamic Time Warping.
  - [tools.py](./tools.py) Utilitary functions for retrieving and plotting time series.

# References

<a id="references"></a>

[Atkinson2012]: P. M. Atkinson, C. Jeganathan, J. Dash, and C. Atzberger, “Inter-comparison of four models for smoothing satellite sensor time-series data to estimate vegetation phenology,” Remote Sens. Environ., vol. 123, pp. 400–417, Aug. 2012.

[Eilers2003]: Paul H. C. Eilers. "A Perfect Smoother". Analytical Chemistry, 2003, 75 (14), pp 3631–3636.


[Vinhas2016]: L. Vinhas; G. R. Queiroz; K. R. Ferreira; Camara, G. [Web Services for Big Earth Observation Data](http://urlib.net/8JMKD3MGP3W34P/3N2U9JL). In: BRAZILIAN SYMPOSIUM ON GEOINFORMATICS, 17. (GEOINFO), 2016, Campos do Jordão, SP. Proceedings... 2016.